# Preparam el notebook

In [5]:
import os
import time
import itertools

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import image
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from skimage.feature import hog
from skimage import io
from skimage.transform import rotate, resize
from skimage.util import random_noise
from scipy import ndimage

# Llegim totes les imatges del dataset

In [2]:
def get_image_files_by_folder(root_folder, image_extensions=('jpg', 'jpeg', 'png', 'gif', 'bmp')):
    image_files_by_folder = {}

    # Walk through the root folder and its subdirectories
    for folder_path, _, files in os.walk(root_folder):
        folder_name = os.path.basename(folder_path)
        image_files = []

        for file in files:
            file_extension = file.lower().split('.')[-1]
            if file_extension in image_extensions:
                image_files.append(os.path.join(folder_path, file))

        if image_files:
            image_files_by_folder[folder_name] = image_files

    return image_files_by_folder

In [3]:
train_dataset_path = "data/train"
images_to_load_train = get_image_files_by_folder(train_dataset_path)
images_to_load_train.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

In [4]:
test_dataset_path = "data/test"
images_to_load_test = get_image_files_by_folder(test_dataset_path)
images_to_load_test.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

# Cream el dataframe

In [33]:
def read_and_flatten_images(image_list):
    start_time = time.time()
    image_data = []
    
    for image_path in image_list:
        img = io.imread(image_path, as_gray=True)
        img = resize(img, (200, 200))
        pca_img = pca_reduction(img)
        hog_vectors = hog(pca_img, orientations=9, pixels_per_cell=(20, 20),
                          cells_per_block=(10, 10), channel_axis=None)
        image_data.append(hog_vectors)
    
    end_time = time.time()
    print("Read + HOG + PCA time:", end_time - start_time, "seconds")

    return image_data

def pca_reduction(input_data):
    pca = PCA(200)
    pca.fit(input_data)
    pca_data = pca.transform(input_data)
    return pca_data

def create_dataframe_from_categories(images_to_load):
    df = pd.DataFrame()
    for label, image_list in images_to_load.items():
        image_data = read_and_flatten_images(images_to_load[label])
        next_part = pd.DataFrame(image_data)
        next_part['label'] = label
        df = pd.concat([df, next_part], ignore_index=True)
    return df

In [34]:
train_df = create_dataframe_from_categories(images_to_load_train)
test_df = create_dataframe_from_categories(images_to_load_test)

Read + HOG + PCA time: 1.0661096572875977 seconds
Read + HOG + PCA time: 1.0869905948638916 seconds
Read + HOG + PCA time: 1.1134445667266846 seconds
Read + HOG + PCA time: 1.0809803009033203 seconds
Read + HOG + PCA time: 1.086987018585205 seconds
Read + HOG + PCA time: 1.111008882522583 seconds
Read + HOG + PCA time: 1.149423360824585 seconds
Read + HOG + PCA time: 1.0771920680999756 seconds
Read + HOG + PCA time: 1.098134994506836 seconds
Read + HOG + PCA time: 1.049088954925537 seconds
Read + HOG + PCA time: 1.1361920833587646 seconds
Read + HOG + PCA time: 1.1203498840332031 seconds
Read + HOG + PCA time: 1.2086846828460693 seconds
Read + HOG + PCA time: 1.2181663513183594 seconds
Read + HOG + PCA time: 1.2240302562713623 seconds
Read + HOG + PCA time: 1.32090425491333 seconds
Read + HOG + PCA time: 2.9699416160583496 seconds
Read + HOG + PCA time: 2.587120532989502 seconds
Read + HOG + PCA time: 1.7509446144104004 seconds
Read + HOG + PCA time: 2.3357062339782715 seconds
Read + H

In [35]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,label
0,0.233947,0.117094,0.041979,0.047137,0.065039,0.049904,0.045960,0.054702,0.174843,0.034666,...,0.000213,0.000181,0.000137,0.000122,0.000111,0.000164,0.000185,0.000101,0.000169,bedroom
1,0.228444,0.020994,0.006660,0.004593,0.013661,0.007352,0.013978,0.024262,0.194555,0.014712,...,0.000133,0.000094,0.000093,0.000106,0.000078,0.000123,0.000085,0.000107,0.000096,bedroom
2,0.218880,0.036662,0.012549,0.009126,0.015933,0.008907,0.011023,0.034341,0.215594,0.033101,...,0.000322,0.000235,0.000140,0.000207,0.000210,0.000297,0.000241,0.000265,0.000264,bedroom
3,0.192970,0.004613,0.000687,0.001446,0.003915,0.000537,0.001579,0.003236,0.205453,0.017466,...,0.000261,0.000134,0.000135,0.000160,0.000155,0.000165,0.000117,0.000130,0.000197,bedroom
4,0.150796,0.010851,0.004438,0.006616,0.014094,0.003776,0.004975,0.015754,0.109821,0.019198,...,0.000051,0.000042,0.000036,0.000043,0.000045,0.000039,0.000031,0.000027,0.000031,bedroom


In [36]:
train_df.shape

(1500, 901)

In [37]:
test_df.shape

(2985, 901)

# Cream els conjunts de test i d'entrenament

In [38]:
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Entrenam el model

In [39]:
svm = SVC(C=1.0, kernel='poly', random_state=42)
start_time = time.time()
svm.fit(X_train, y_train)
print("fit time : ", time.time() - start_time, " seconds")

fit time :  0.3573274612426758  seconds


# Donam una predicció

In [40]:
y_predicted = svm.predict(X_test)

print(f"La precisició es de: {accuracy_score(y_test, y_predicted)}")

La precisició es de: 0.5122278056951424
